In [2]:
pip install scikeras


In [1]:
# Import necessary libraries for data handling, preprocessing,
# and building a deep learning model
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras import layers, optimizers
import keras
from sklearn.preprocessing import StandardScaler
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV


In [2]:
# Load the dataset and preview the first few rows

df = pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
# Separate features and target variable, then standardize the feature data

X = df.drop('Outcome', axis=1)
y = df['Outcome']

scaler = StandardScaler()
X = scaler.fit_transform(X)


In [4]:
# Build a Sequential neural network model
# Layer 1: Dense layer with 32 neurons, ReLU activation, and input shape matching feature count
# Layer 2: Dense layer with 16 neurons and ReLU activation
# Output Layer: Dense layer with 1 neuron and sigmoid activation for binary classification


model = Sequential()
model.add(layers.Dense(32, activation='relu', input_shape=(X.shape[1],)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)

In [10]:
# check model accuracy
loss, accuracy = model.evaluate(X_train, y_train)
print(f'Accuracy: {accuracy}')

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8587 - loss: 0.3411  
Accuracy: 0.8583061695098877


# Hyper Parameter Tuning With Keras-tuner

In [11]:
# lets install keras tuner for parameter tuning
!pip install keras-tuner -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 8.9 MB/s eta 0:00:00


In [12]:
# Import Keras Tuner for hyperparameter optimization

import keras_tuner

In [13]:
#  Define a tunable neural network model for hyperparameter optimization using Keras Tuner
# - First hidden layer: tunable units (16–128) with ReLU activation
# - Additional hidden layers: tunable count (2–5), units (8–64), dropout rate (0.0–0.5)
# - Output layer: 1 neuron with sigmoid activation for binary classification
# - Optimizer: Adam with tunable learning rate (1e-4 to 1e-1)

def model_tuning(hp):
    model = Sequential()

    # First hidden layer
    model.add(layers.Dense(
        units=hp.Int('units', min_value=16, max_value=128, step=16),
        activation='relu',
        input_shape=(X.shape[1],)
    ))

    # Tunable number of additional hidden layers
    for i in range(hp.Int('num_layers', 2, 5)):
        model.add(layers.Dense(
            units=hp.Int(f'units_{i}', min_value=8, max_value=64, step=8),
            kernel_initializer=keras.initializers.HeNormal(),
            activation='relu'
        ))
        model.add(layers.Dropout(rate=hp.Float('dropout_' + str(i), 0.0, 0.5, step=0.1)))

    # Output layer
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=keras.optimizers.Adam(
            learning_rate=hp.Float('lr', min_value=1e-4, max_value=1e-1, sampling="log")
        ),

        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model


In [14]:
# Set up Keras Tuner (Random Search) for hyperparameter tuning

tuner = keras_tuner.RandomSearch(
    model_tuning,
    objective='val_accuracy',
    max_trials=10,                  # More attempts
    executions_per_trial=3,         # Average results
    overwrite=True,
    directory='project_5',
    project_name='diabetes_ann'
)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
# Run hyperparameter search
tuner.search(X_train, y_train, epochs=100, validation_data=(X_test, y_test),verbose=0)

In [18]:
# Display tuning results summary
tuner.results_summary()

Results summary
Results in project_5/diabetes_ann
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 00 summary
Hyperparameters:
units: 128
num_layers: 3
units_0: 48
dropout_0: 0.30000000000000004
units_1: 48
dropout_1: 0.2
lr: 0.010679693834169201
units_2: 8
dropout_2: 0.0
Score: 0.7965367833773295

Trial 03 summary
Hyperparameters:
units: 32
num_layers: 2
units_0: 16
dropout_0: 0.2
units_1: 24
dropout_1: 0.30000000000000004
lr: 0.0017119224401432604
units_2: 64
dropout_2: 0.0
units_3: 48
dropout_3: 0.1
units_4: 48
dropout_4: 0.4
Score: 0.7878787716229757

Trial 05 summary
Hyperparameters:
units: 112
num_layers: 5
units_0: 24
dropout_0: 0.30000000000000004
units_1: 16
dropout_1: 0.2
lr: 0.00038942952803452897
units_2: 16
dropout_2: 0.4
units_3: 48
dropout_3: 0.2
units_4: 48
dropout_4: 0.1
Score: 0.7857142686843872

Trial 06 summary
Hyperparameters:
units: 48
num_layers: 2
units_0: 32
dropout_0: 0.30000000000000004
units_1: 16
dropout_1: 0.4
lr: 0.00842536297

# Hyper Parameter Tuning With Grid Seach CV

In [6]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras import Sequential, layers

# Function to create model
def create_model(layer_list, activation):
    model = Sequential()
    for i, nodes in enumerate(layer_list):
      # if layer i = 0 then it will be the input layer
        if i == 0:
            model.add(layers.Dense(nodes, input_dim=X_train.shape[1]))
            model.add(layers.Activation(activation))
            model.add(layers.Dropout(0.3))
        else:
            model.add(layers.Dense(nodes))
            model.add(layers.Activation(activation))
            model.add(layers.Dropout(0.3))

    model.add(layers.Dense(units=1, kernel_initializer='glorot_uniform', activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [7]:
# model = KerasClassifier(model=create_model, verbose=0)
model = KerasClassifier(model=create_model, verbose=0, random_state=42)

In [11]:
from sklearn.model_selection import GridSearchCV
# from scikeras.wrappers import KerasClassifier
# from tensorflow.keras import Sequential, layers

param_grid = {
    'model__layer_list': [[16,32], [16, 32, 16], [16, 8, 16, 32]],
    'model__activation': ['sigmoid', 'relu'],
    'batch_size': [128, 256],
    'epochs': [30]
}


grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')
grid_result = grid.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/lo

In [12]:
# now print best parameters
print(grid_result.best_score_,grid_result.best_params_)

0.7426229508196721 {'batch_size': 128, 'epochs': 30, 'model__activation': 'relu', 'model__layer_list': [16, 32, 16]}
